In [1]:
import os
import os.path
import glob
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix,accuracy_score

import matplotlib.pyplot as plt
import matplotlib.cm as colormap
plt.rcParams['image.cmap'] = 'Pastel1'

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,Flatten,Dense,Dropout,GlobalAveragePooling2D,Conv2D,MaxPooling2D

Using TensorFlow backend.


In [2]:
imagedir = "retina-dataset"

In [4]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
#X = np.zeros((num_samples, width, height, channels))
cnt = 0
#list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        #list_paths.append(os.path.join(os.getcwd(),img_file))
        #img = image.load_img(img_file, target_size=(224, 224))
        #x = image.img_to_array(img)
        #x = np.expand_dims(x, axis=0)
        #x = preprocess_input(x)
        #X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         Exudate	Number of images: 67600
Label: 1	Family:     Non_Exudate	Number of images: 70043
Processing images ...
Images processed: 137643


In [5]:
#X.shape

In [6]:
# Encoding classes (y) into integers (y_encoded) and then generating one-hot-encoding (Y)
encoder = LabelEncoder()
encoder.fit(y)
y_encoded = encoder.transform(y)
Y = np_utils.to_categorical(y_encoded)

In [7]:
# Creating base_model (ResNet50 notop)
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = ResNet50(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [8]:
filename = 'retina-resnet50features-avgpool.npy'
if os.path.exists(filename):
    print("Loading ResNet50 extracted features from %s ..." %(filename))
    resnet50features = np.load(filename)
else:
    print("Extracting features from ResNet50 layers ...")
    resnet50features = base_model.predict(X)
    print("Saving ResNet50 extracted features into %s ..." %(filename))
    np.save(filename, resnet50features)

Loading ResNet50 extracted features from retina-resnet50features-avgpool.npy ...


In [9]:
resnet50features.shape

(137643, 2048)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedShuffleSplit(n_splits=kfold, test_size=0.15, random_state=1)
skfind = [None] * kfold  # skfind[i][0] -> train indices, skfind[i][1] -> test indices
cnt = 0                                              
for index in skf.split(resnet50features, y):         
    skfind[cnt] = index                                                 
    cnt += 1 

In [ ]:
# Training top_model and saving min training loss weights
num_epochs=20000
history = []
conf_mat = np.zeros((len(list_fams),len(list_fams))) # Initializing the Confusion Matrix
early_stopping = EarlyStopping(verbose=1, patience=20, monitor='val_loss')
checkpointer = ModelCheckpoint(filepath='retina-0.85train_0.15test-max_val_acc-resnet50-Adam-20000epochs_earlystop.h5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
callbacks_list = [early_stopping, checkpointer]
for i in range(kfold):
    train_indices = skfind[i][0]
    test_indices = skfind[i][1]
    X_train = resnet50features[train_indices]
    Y_train = Y[train_indices]
    X_test = resnet50features[test_indices]
    Y_test = Y[test_indices]
    y_test = y[test_indices]

    top_input = Input(shape=resnet50features.shape[1:])
    predict = Dense(num_classes, activation='softmax', name='predictions')(top_input)
    top_model = Model(input=top_input, output=predict)                            
    top_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    start = time.time()
    h = top_model.fit(X_train, Y_train, validation_split=0.15, epochs=num_epochs, batch_size=X_train.shape[0], verbose=1, callbacks=callbacks_list)
    end = time.time()
    history.append(h)
    
    y_prob = top_model.predict(X_test, verbose=0)  # Testing
    y_pred = np.argmax(y_prob, axis=1)
    print("[%d] Test acurracy: %.4f (%.4f s)" %(i,accuracy_score(y_test,y_pred),end-start))
    cm = confusion_matrix(y_test,y_pred)  # Compute confusion matrix for this fold
    conf_mat = conf_mat + cm  # Compute global confusion matrix

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("pr..., inputs=Tensor("in...)`


Train on 99446 samples, validate on 17550 samples
Epoch 1/20000
99446/99446 [==============================] - 16s - loss: 0.7914 - acc: 0.4847 - val_loss: 0.6866 - val_acc: 0.5152
Epoch 2/20000
99446/99446 [==============================] - 7s - loss: 0.6869 - acc: 0.5155 - val_loss: 0.6469 - val_acc: 0.6618
Epoch 3/20000
99446/99446 [==============================] - 10s - loss: 0.6461 - acc: 0.6664 - val_loss: 0.6397 - val_acc: 0.6369
Epoch 4/20000
99446/99446 [==============================] - 10s - loss: 0.6382 - acc: 0.6360 - val_loss: 0.6318 - val_acc: 0.6263
Epoch 5/20000
99446/99446 [==============================] - 11s - loss: 0.6300 - acc: 0.6288 - val_loss: 0.6117 - val_acc: 0.6538
Epoch 6/20000
99446/99446 [==============================] - 11s - loss: 0.6100 - acc: 0.6582 - val_loss: 0.5842 - val_acc: 0.7031
Epoch 7/20000
99446/99446 [==============================] - 11s - loss: 0.5826 - acc: 0.7056 - val_loss: 0.5569 - val_acc: 0.7450
Epoch 8/20000
99446/99446 [=======

99446/99446 [==============================] - 9s - loss: 0.3776 - acc: 0.8263 - val_loss: 0.3779 - val_acc: 0.8279
Epoch 64/20000
99446/99446 [==============================] - 9s - loss: 0.3763 - acc: 0.8272 - val_loss: 0.3767 - val_acc: 0.8287
Epoch 65/20000
99446/99446 [==============================] - 7s - loss: 0.3750 - acc: 0.8280 - val_loss: 0.3754 - val_acc: 0.8293
Epoch 66/20000
99446/99446 [==============================] - 10s - loss: 0.3738 - acc: 0.8289 - val_loss: 0.3743 - val_acc: 0.8303
Epoch 67/20000
99446/99446 [==============================] - 9s - loss: 0.3726 - acc: 0.8295 - val_loss: 0.3731 - val_acc: 0.8309
Epoch 68/20000
99446/99446 [==============================] - 8s - loss: 0.3714 - acc: 0.8302 - val_loss: 0.3719 - val_acc: 0.8321
Epoch 69/20000
99446/99446 [==============================] - 7s - loss: 0.3702 - acc: 0.8311 - val_loss: 0.3708 - val_acc: 0.8326
Epoch 70/20000
99446/99446 [==============================] - 8s - loss: 0.3690 - acc: 0.8317 - v

99446/99446 [==============================] - 9s - loss: 0.3193 - acc: 0.8664 - val_loss: 0.3211 - val_acc: 0.8648
Epoch 126/20000
99446/99446 [==============================] - 6s - loss: 0.3186 - acc: 0.8668 - val_loss: 0.3204 - val_acc: 0.8652
Epoch 127/20000
99446/99446 [==============================] - 4s - loss: 0.3179 - acc: 0.8671 - val_loss: 0.3197 - val_acc: 0.8659
Epoch 128/20000
99446/99446 [==============================] - 6s - loss: 0.3172 - acc: 0.8674 - val_loss: 0.3190 - val_acc: 0.8660
Epoch 129/20000
99446/99446 [==============================] - 4s - loss: 0.3166 - acc: 0.8676 - val_loss: 0.3184 - val_acc: 0.8663
Epoch 130/20000
99446/99446 [==============================] - 7s - loss: 0.3159 - acc: 0.8678 - val_loss: 0.3177 - val_acc: 0.8668
Epoch 131/20000
99446/99446 [==============================] - 6s - loss: 0.3152 - acc: 0.8681 - val_loss: 0.3171 - val_acc: 0.8674
Epoch 132/20000
99446/99446 [==============================] - 6s - loss: 0.3145 - acc: 0.86

99446/99446 [==============================] - 4s - loss: 0.2849 - acc: 0.8851 - val_loss: 0.2872 - val_acc: 0.8856
Epoch 188/20000
99446/99446 [==============================] - 7s - loss: 0.2844 - acc: 0.8854 - val_loss: 0.2867 - val_acc: 0.8858
Epoch 189/20000
99446/99446 [==============================] - 5s - loss: 0.2840 - acc: 0.8856 - val_loss: 0.2863 - val_acc: 0.8858
Epoch 190/20000
99446/99446 [==============================] - 6s - loss: 0.2836 - acc: 0.8859 - val_loss: 0.2859 - val_acc: 0.8859
Epoch 191/20000
99446/99446 [==============================] - 5s - loss: 0.2831 - acc: 0.8861 - val_loss: 0.2855 - val_acc: 0.8862
Epoch 192/20000
99446/99446 [==============================] - 6s - loss: 0.2827 - acc: 0.8862 - val_loss: 0.2850 - val_acc: 0.8863
Epoch 193/20000
99446/99446 [==============================] - 5s - loss: 0.2823 - acc: 0.8863 - val_loss: 0.2846 - val_acc: 0.8867
Epoch 194/20000
99446/99446 [==============================] - 6s - loss: 0.2818 - acc: 0.88

...

99446/99446 [==============================] - 5s - loss: 0.1199 - acc: 0.9529 - val_loss: 0.1286 - val_acc: 0.9489
Epoch 4348/20000
99446/99446 [==============================] - 7s - loss: 0.1199 - acc: 0.9529 - val_loss: 0.1286 - val_acc: 0.9489
Epoch 4349/20000
99446/99446 [==============================] - 4s - loss: 0.1198 - acc: 0.9529 - val_loss: 0.1285 - val_acc: 0.9489
Epoch 4350/20000
99446/99446 [==============================] - 6s - loss: 0.1198 - acc: 0.9529 - val_loss: 0.1285 - val_acc: 0.9489
Epoch 4351/20000
99446/99446 [==============================] - 5s - loss: 0.1198 - acc: 0.9529 - val_loss: 0.1285 - val_acc: 0.9489
Epoch 4352/20000
99446/99446 [==============================] - 8s - loss: 0.1198 - acc: 0.9529 - val_loss: 0.1285 - val_acc: 0.9489
Epoch 4353/20000
99446/99446 [==============================] - 5s - loss: 0.1198 - acc: 0.9529 - val_loss: 0.1285 - val_acc: 0.9489
Epoch 4354/20000
99446/99446 [==============================] - 6s - loss: 0.1198 - ac

99446/99446 [==============================] - 8s - loss: 0.1195 - acc: 0.9529 - val_loss: 0.1283 - val_acc: 0.9489
Epoch 4409/20000
99446/99446 [==============================] - 4s - loss: 0.1195 - acc: 0.9529 - val_loss: 0.1283 - val_acc: 0.9489
Epoch 4410/20000
99446/99446 [==============================] - 7s - loss: 0.1195 - acc: 0.9529 - val_loss: 0.1283 - val_acc: 0.9490
Epoch 4411/20000
99446/99446 [==============================] - 5s - loss: 0.1195 - acc: 0.9529 - val_loss: 0.1283 - val_acc: 0.9490
Epoch 4412/20000
99446/99446 [==============================] - 6s - loss: 0.1195 - acc: 0.9529 - val_loss: 0.1283 - val_acc: 0.9490
Epoch 4413/20000
99446/99446 [==============================] - 5s - loss: 0.1195 - acc: 0.9529 - val_loss: 0.1283 - val_acc: 0.9490
Epoch 4414/20000
99446/99446 [==============================] - 8s - loss: 0.1195 - acc: 0.9529 - val_loss: 0.1283 - val_acc: 0.9490
Epoch 4415/20000
99446/99446 [==============================] - 5s - loss: 0.1194 - ac

99446/99446 [==============================] - 6s - loss: 0.1191 - acc: 0.9530 - val_loss: 0.1281 - val_acc: 0.9490
Epoch 4470/20000
99446/99446 [==============================] - 8s - loss: 0.1191 - acc: 0.9530 - val_loss: 0.1281 - val_acc: 0.9490
Epoch 4471/20000
99446/99446 [==============================] - 6s - loss: 0.1191 - acc: 0.9530 - val_loss: 0.1281 - val_acc: 0.9490
Epoch 4472/20000
99446/99446 [==============================] - 8s - loss: 0.1191 - acc: 0.9530 - val_loss: 0.1281 - val_acc: 0.9490
Epoch 4473/20000
99446/99446 [==============================] - 5s - loss: 0.1191 - acc: 0.9530 - val_loss: 0.1281 - val_acc: 0.9490
Epoch 4474/20000
99446/99446 [==============================] - 7s - loss: 0.1191 - acc: 0.9530 - val_loss: 0.1281 - val_acc: 0.9490
Epoch 4475/20000
99446/99446 [==============================] - 6s - loss: 0.1191 - acc: 0.9530 - val_loss: 0.1281 - val_acc: 0.9490
Epoch 4476/20000
99446/99446 [==============================] - 7s - loss: 0.1191 - ac

99446/99446 [==============================] - 7s - loss: 0.1188 - acc: 0.9531 - val_loss: 0.1279 - val_acc: 0.9491
Epoch 4531/20000
99446/99446 [==============================] - 6s - loss: 0.1188 - acc: 0.9531 - val_loss: 0.1279 - val_acc: 0.9491
Epoch 4532/20000
99446/99446 [==============================] - 7s - loss: 0.1188 - acc: 0.9531 - val_loss: 0.1279 - val_acc: 0.9491
Epoch 4533/20000
99446/99446 [==============================] - 7s - loss: 0.1188 - acc: 0.9532 - val_loss: 0.1279 - val_acc: 0.9491
Epoch 4534/20000
99446/99446 [==============================] - 6s - loss: 0.1188 - acc: 0.9532 - val_loss: 0.1279 - val_acc: 0.9491
Epoch 4535/20000
99446/99446 [==============================] - 7s - loss: 0.1187 - acc: 0.9532 - val_loss: 0.1279 - val_acc: 0.9492
Epoch 4536/20000
99446/99446 [==============================] - 7s - loss: 0.1187 - acc: 0.9532 - val_loss: 0.1279 - val_acc: 0.9491
Epoch 4537/20000
99446/99446 [==============================] - 7s - loss: 0.1187 - ac

99446/99446 [==============================] - 4s - loss: 0.1184 - acc: 0.9532 - val_loss: 0.1277 - val_acc: 0.9489
Epoch 4592/20000
99446/99446 [==============================] - 4s - loss: 0.1184 - acc: 0.9532 - val_loss: 0.1277 - val_acc: 0.9490
Epoch 4593/20000
99446/99446 [==============================] - 4s - loss: 0.1184 - acc: 0.9533 - val_loss: 0.1277 - val_acc: 0.9491
Epoch 4594/20000
99446/99446 [==============================] - 3s - loss: 0.1184 - acc: 0.9533 - val_loss: 0.1277 - val_acc: 0.9491
Epoch 4595/20000
99446/99446 [==============================] - 5s - loss: 0.1184 - acc: 0.9533 - val_loss: 0.1277 - val_acc: 0.9491
Epoch 4596/20000
99446/99446 [==============================] - 3s - loss: 0.1184 - acc: 0.9533 - val_loss: 0.1277 - val_acc: 0.9491
Epoch 4597/20000
99446/99446 [==============================] - 4s - loss: 0.1184 - acc: 0.9533 - val_loss: 0.1277 - val_acc: 0.9490
Epoch 4598/20000
99446/99446 [==============================] - 4s - loss: 0.1184 - ac

In [ ]:
# Computing the average accuracy
avg_acc = np.trace(conf_mat)/np.sum(conf_mat)
print("Average acurracy: %.4f" %(avg_acc))

In [ ]:
def plot_acc(history):
    figure = plt.gcf()
    figure.set_size_inches(15, 5)
    ax = plt.subplot()
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    colors = iter(colormap.gist_rainbow(np.linspace(0, 1, len(history))))
    for i in range(len(history)):
        color=next(colors)
        plt.plot(history[i].history['acc'], label='Train '+str(i), color=color, linestyle = 'solid')
        plt.plot(history[i].history['val_acc'], label='Test '+str(i), color=color, linestyle = 'dotted')
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,0.0,1.0))
    plt.legend()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()

In [ ]:
def plot_loss(history):
    figure = plt.gcf()
    figure.set_size_inches(20, 8)
    ax = plt.subplot()
    plt.title('Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    colors = iter(colormap.gist_rainbow(np.linspace(0, 1, len(history))))
    for i in range(len(history)):
        color=next(colors)
        plt.plot(history[i].history['loss'], label='Train '+str(i), color=color, linestyle = 'solid')
        plt.plot(history[i].history['val_loss'], label='Test '+str(i), color=color, linestyle = 'dotted')
    plt.legend()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()

In [ ]:
plot_acc(history)

In [ ]:
plot_loss(history)

In [ ]:
print("Plotting the confusion matrix")
figure = plt.gcf()
figure.set_size_inches(5, 4)
plt.imshow(conf_mat,interpolation='nearest')
for row in range(len(list_fams)):
    for col in range(len(list_fams)):
        plt.annotate(str(int(conf_mat[row][col])),xy=(col,row),ha='center',va='center')
plt.xticks(range(len(list_fams)),list_fams,rotation=90,fontsize=10)
plt.yticks(range(len(list_fams)),list_fams,fontsize=10)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
print("Plotting the confusion matrix normalized")
conf_mat_norm = conf_mat/np.sum(conf_mat,axis=1)  # Normalizing the confusion matrix
conf_mat_norm = np.around(conf_mat_norm,decimals=4)  # rounding to display in figure
figure = plt.gcf()
figure.set_size_inches(5, 4)
plt.imshow(conf_mat_norm,interpolation='nearest')
for row in range(len(list_fams)):
    for col in range(len(list_fams)):
        plt.annotate(str(conf_mat_norm[row][col]),xy=(col,row),ha='center',va='center')
plt.xticks(range(len(list_fams)),list_fams,rotation=90,fontsize=10)
plt.yticks(range(len(list_fams)),list_fams,fontsize=10)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()